## Расчёт уровней TTC и PIT для калибровки модели
**Цель**: Рассчитать целевые уровни дефолтов TTC (долгосрочный средний) и PIT (point in time) для калибровки модели кредитного риска на основе фактических данных портфеля.

**Prerequisites**:
- Здоровый корпоративный портфель;
- Данные о динамике ВВП по кварталам;
- Прописать параметры расчёта в конфигурации в данном ноутбуке.

**Результат**: Конфигурационный файл config.yaml с рассчитанными уровнями TTC и PIT.

-----------
**Подробнее**:

Ноутбук рассчитывает фактические уровни дефолтов в портфеле для правильной калибровки модели.
Сначала определяются ежемесячные уровни дефолтов — для каждого месячного среза портфеля рассчитывается процент дефолтных клиентов. Затем данные агрегируются по кварталам для сглаживания волатильности.
Из расчёта исключаются нерепрезентативные периоды.

Дополнительно строится визуализация с наложением динамики ВВП, показывающая корреляцию экономических циклов и уровней дефолтов.

### Библиотеки

In [1]:
import numpy as np 
import pandas as pd
import warnings
from dataclasses import dataclass, field, asdict
from typing import List, Optional, Tuple, NamedTuple

import yaml

import plotly.graph_objects as go
from plotly.subplots import make_subplots
# from funcs_central_tendencies import *

In [2]:
warnings.filterwarnings("ignore")

# Настройка отображения
# plt.rcParams['figure.figsize'] = (15, 10)
# plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)


### Конфигурации

In [3]:
with open('../CONFIGS.yaml', 'r') as file:
    CONFIG = yaml.safe_load(file)

PATHS = CONFIG['data_paths']

In [4]:
class Period(NamedTuple):
    year: int
    quarter: int

In [5]:
filepath = {
      'gdp_data'      : PATHS['other']['gpd_data']
    , 'portfolio'     : PATHS['portfolio']['healthy']
    , 'config'        : PATHS['other']['this_config']
}

In [6]:
calib_params = CONFIG['calibration_params']

analysis_start_date   = pd.to_datetime(calib_params['analysis_period'][0], format="%Y-%m-%d")
analysis_end_date     = pd.to_datetime(calib_params['analysis_period'][1], format="%Y-%m-%d")
min_year_for_analysis = analysis_start_date.year

unripe_period_start   = Period(calib_params['unripe_period_start'][0], calib_params['unripe_period_start'][1])

excluded_periods = [  Period(y, q) for y,q in calib_params['excluded_quarters']] 

pit_periods = [Period(y, q) for y,q in calib_params['pit_quarters']]

print(f'analysis_start_date:\n\t {analysis_start_date}')
print(f'analysis_end_date: \n\t{analysis_end_date}')
print(f'min_year_for_analysis:\n\t {min_year_for_analysis}')
print(f'unripe_period_start: \n\t{unripe_period_start}')
print(f'excluded_periods: \n\t{excluded_periods}')
print(f'pit_periods: \n\t{pit_periods}')

analysis_start_date:
	 2018-01-01 00:00:00
analysis_end_date: 
	2024-08-01 00:00:00
min_year_for_analysis:
	 2018
unripe_period_start: 
	Period(year=2025, quarter=3)
excluded_periods: 
	[Period(year=2018, quarter=1), Period(year=2018, quarter=2), Period(year=2024, quarter=3)]
pit_periods: 
	[Period(year=2023, quarter=3), Period(year=2023, quarter=4), Period(year=2024, quarter=1), Period(year=2024, quarter=2)]


### Загрузка данных

In [7]:
portfolio_data = pd.read_csv(filepath['portfolio'])
portfolio_data['date'] = pd.to_datetime(
    portfolio_data['date'], 
    format='%Y-%m-%d'
)

In [8]:
gdp_data = pd.read_excel(filepath['gdp_data'])
# Преобразование GDP из строкового формата в числовой
gdp_data['gdp_cmltv'] = gdp_data['gdp_cmltv'].str.replace(',', '.').astype(float)

# # Извлечение года и квартала из year_quarter
gdp_data['year']    = gdp_data['year_quarter'].str.extract(r'(\d{4})')[0].astype(int)
gdp_data['quarter'] = gdp_data['year_quarter'].str.extract(r'Q(\d)')[0].astype(int)

gdp_data = gdp_data[['year', 'quarter', 'gdp_cmltv']]

In [9]:
print(f"Загружено {len(portfolio_data)} записей портфеля")
print(f"Загружено {len(gdp_data)} записей ВВП")

Загружено 30459 записей портфеля
Загружено 62 записей ВВП


Восстановление чистых квартальных значений ВВП

In [10]:
def restore_quarterly_gdp(df: pd.DataFrame) -> pd.DataFrame:
    """
    Восстанавливает реальные квартальные значения ВВП из накопительных данных.
    """
    df = df.sort_values(['year', 'quarter'])
    
    def process_year(year_df):
        """Обработка данных одного года"""
        year_df = year_df.sort_values('quarter').copy()
        quarterly_values = []
        
        for q in range(1, 5):
            q_data = year_df[year_df['quarter'] == q]
            if q_data.empty:
                continue
                
            cumulative_value = q_data['gdp_cmltv'].values[0]
            
            if q == 1:
                quarterly_values.append(cumulative_value)
            elif q == 2:
                q_value = cumulative_value * 2 - quarterly_values[0]
                quarterly_values.append(q_value)
            elif q == 3:
                q_value = cumulative_value * 3 - sum(quarterly_values[:2])
                quarterly_values.append(q_value)
            elif q == 4:
                q_value = cumulative_value * 4 - sum(quarterly_values[:3])
                quarterly_values.append(q_value)
        
        year_df['gdp_quarterly'] = quarterly_values[:len(year_df)]
        return year_df
    
    # Обработка каждого года
    result_dfs = []
    for year in df['year'].unique():
        # year_mask = df['year'] == year
        year_data = df.query(f'year == {year}').copy()
        
        if year_data['gdp_cmltv'].isna().all():
            year_data['gdp_quarterly'] = np.nan
            result_dfs.append(year_data)
            continue
            
        processed_year = process_year(year_data)
        result_dfs.append(processed_year)
    
    result = pd.concat(result_dfs, ignore_index=True)
    result = result.sort_values(['year', 'quarter'])
    result['gdp_quarterly'] = result['gdp_quarterly'].round(2)
    
    return result

In [11]:
gdp_data = restore_quarterly_gdp(gdp_data)

gdp_data

,year,quarter,gdp_cmltv,gdp_quarterly
0,2010,1,107.1,107.1
1,2010,2,108.0,108.9
2,2010,3,107.5,106.5
3,2010,4,107.3,106.7
4,2011,1,106.6,106.6
...,...,...,...,...
57,2024,2,103.2,102.6
58,2024,3,104.1,105.9
59,2024,4,104.8,106.9
60,2025,1,105.6,105.6


#### Ежемесячная статистика

In [12]:
monthly_stats = portfolio_data.copy()
monthly_stats = monthly_stats.query('date >=@analysis_start_date & date <= @analysis_end_date')
monthly_stats = monthly_stats\
                .groupby('date')['target']\
                .agg(  total_clients='count'
                     , number_of_defaults='sum'
                     , monthly_dr='mean')\
               .reset_index()
monthly_stats['year'] = monthly_stats.date.dt.year
monthly_stats['quarter'] = monthly_stats.date.dt.quarter

#### Квартальная статистика

In [13]:
quarterly_stats = \
        monthly_stats\
        .groupby(['year', 'quarter'], as_index=False)\
        .agg({
            'monthly_dr': 'mean',
            'total_clients': 'sum'
        })\
        .rename(columns={'monthly_dr':'quarterly_dr'})
    

In [14]:
quarterly_stats_filtered = quarterly_stats.query('year >= @min_year_for_analysis')

for period in excluded_periods:
    mask = (quarterly_stats_filtered['year'] == period.year) & (quarterly_stats_filtered['quarter'] == period.quarter)
    quarterly_stats_filtered.loc[mask, 'quarterly_dr'] = np.nan

ripe_years = quarterly_stats_filtered['year'] < unripe_period_start.year
unripe_year_ripe_quarters = (quarterly_stats_filtered['year'] == unripe_period_start.year) & (quarterly_stats_filtered['quarter'] < unripe_period_start.quarter)

quarterly_stats_filtered = quarterly_stats_filtered[ripe_years | unripe_year_ripe_quarters]
quarterly_stats_filtered = quarterly_stats_filtered.dropna()

In [15]:
quarterly_stats_filtered

,year,quarter,quarterly_dr,total_clients
2,2018,3,0.047154,1039
3,2018,4,0.040247,1044
4,2019,1,0.042338,1063
5,2019,2,0.051705,1121
6,2019,3,0.063468,1135
7,2019,4,0.080417,1144
8,2020,1,0.085480,1123
9,2020,2,0.065031,1136
10,2020,3,0.042649,1171
11,2020,4,0.031325,1213


#### Метрики Центральной Тенденции

In [16]:
ttc_level = float(quarterly_stats_filtered['quarterly_dr'].mean())
ttc_level

0.05504190055045175

In [17]:
# PIT - средний уровень в стрессовый период
print(pit_periods)
pit_mask = quarterly_stats_filtered.apply(
    lambda row: Period(int(row['year']), int(row['quarter'])) in pit_periods,
    axis=1
)
pit_data = quarterly_stats_filtered[pit_mask]
display(pit_data)
pit_level = float(pit_data['quarterly_dr'].mean())
pit_level

[Period(year=2023, quarter=3), Period(year=2023, quarter=4), Period(year=2024, quarter=1), Period(year=2024, quarter=2)]


,year,quarter,quarterly_dr,total_clients
22,2023,3,0.053856,687
23,2023,4,0.059967,651
24,2024,1,0.060121,666
25,2024,2,0.037012,675


0.05273895123991376

In [18]:
annual_drs = (
        quarterly_stats_filtered
        .groupby('year')['quarterly_dr']
        .mean()
        .astype(float)
        .to_dict()
    )
annual_drs

{2018: 0.04370056450116898,
 2019: 0.05948206711312044,
 2020: 0.05612097800571561,
 2021: 0.04052443466994591,
 2022: 0.07823005900029882,
 2023: 0.04976042653970924,
 2024: 0.04856631144667195}

In [19]:
print(f"\n{'='*50}")
print(f"РЕЗУЛЬТАТЫ КАЛИБРОВКИ:")
print(f"{'='*50}")
print(f"Уровень TTC (Through-The-Cycle): {ttc_level*100:.2f}%")
print(f"Уровень PIT (Point-In-Time): {pit_level*100:.2f}%")
print(f"Средние годовые уровни дефолта:")
display({ y : round(r*100,2) for y,r in annual_drs.items()})


РЕЗУЛЬТАТЫ КАЛИБРОВКИ:
Уровень TTC (Through-The-Cycle): 5.50%
Уровень PIT (Point-In-Time): 5.27%
Средние годовые уровни дефолта:


{2018: 4.37,
 2019: 5.95,
 2020: 5.61,
 2021: 4.05,
 2022: 7.82,
 2023: 4.98,
 2024: 4.86}

#### Объединение поквартальных данных о дефолтах и ВВП 

In [20]:
defaults_with_gdp = quarterly_stats_filtered.merge(
    gdp_data,
    on=['year', 'quarter'],
    how='left'
)    
defaults_with_gdp

,year,quarter,quarterly_dr,total_clients,gdp_cmltv,gdp_quarterly
0,2018,3,0.047154,1039,104.1,103.9
1,2018,4,0.040247,1044,104.1,104.1
2,2019,1,0.042338,1063,103.8,103.8
3,2019,2,0.051705,1121,104.1,104.4
4,2019,3,0.063468,1135,104.3,104.7
5,2019,4,0.080417,1144,104.5,105.1
6,2020,1,0.085480,1123,102.7,102.7
7,2020,2,0.065031,1136,98.2,93.7
8,2020,3,0.042649,1171,97.2,95.2
9,2020,4,0.031325,1213,97.5,98.4


### Визуализация

In [21]:
def create_calibration_chart(data_df: pd.DataFrame, 
                            ttc_level: float, 
                            pit_level: float,
                            pit_periods: Period,
                            save_path: Optional[str] = None) -> go.Figure:
    """
    Создание единого графика со всеми данными калибровки
    """
    # Подготовка данных
    data_df = data_df.sort_values(['year', 'quarter'])
    quarters_labels = data_df.apply(lambda row: f"{int(row['year'])} Q{int(row['quarter'])}", axis=1).tolist()
    default_rates = data_df['quarterly_dr'].tolist()
    
    # Создание фигуры с двумя осями Y
    fig = make_subplots(
        specs=[[{"secondary_y": True}]],
        figure=go.Figure()
    )
    
    # 1. Количество компаний (на заднем плане)
    if 'total_clients' in data_df.columns:
        clients_count = data_df['total_clients'].tolist()
        
        fig.add_trace(
            go.Bar(
                x=quarters_labels,
                y=clients_count,
                name='К-во компаний',
                marker=dict(
                    color='grey',
                    opacity=0.4,
                    line=dict(width=0)
                ),
                yaxis='y2',
                hovertemplate='<b>%{x}</b><br>К-во компаний: %{y:.0f}<br><extra></extra>'
            ),
            secondary_y=True
        )
    
    # 2. ВВП линия
    if 'gdp_quarterly' in data_df.columns and not data_df['gdp_quarterly'].isna().all():
        gdp_values = (data_df['gdp_quarterly'] - 100).tolist()
        
        fig.add_trace(
            go.Scatter(
                x=quarters_labels,
                y=gdp_values,
                name='ВВП (отклонение от базы)',
                mode='lines+markers+text',
                line=dict(color='darkblue', width=1),
                marker=dict(
                    size=8,
                    color='darkblue',
                    symbol='square'
                ),
                text=[f'{gdp:.1f}' for gdp in gdp_values],
                textposition='top center',
                textfont=dict(size=9, color='darkblue'),
                yaxis='y1',
                hovertemplate='<b>%{x}</b><br>ВВП отклонение: %{y:.1f}<br><extra></extra>'
            ),
            secondary_y=False
        )
    
    # 3. Основная линия - уровень дефолта
    fig.add_trace(
        go.Scatter(
            x=quarters_labels,
            y=default_rates,
            mode='lines+markers+text',
            name='Уровень дефолта (%)',
            line=dict(color='darkgreen', width=3),
            marker=dict(
                size=8,
                color='darkgreen',
                symbol='circle'
            ),
            text=[f'{rate:.1f}' for rate in default_rates],
            textposition='top center',
            textfont=dict(size=9, color='darkgreen'),
            hovertemplate='<b>%{x}</b><br>Уровень дефолта: %{y:.2f}%<extra></extra>'
        ),
        secondary_y=False
    )
    
    # 4. Линия TTC
    fig.add_trace(
        go.Scatter(
            x=quarters_labels,
            y=[ttc_level] * len(quarters_labels),
            mode='lines',
            name=f'TTC: {ttc_level:.2f}%',
            line=dict(color='red', width=2, dash='dash'),
            hovertemplate=f'TTC: {ttc_level:.2f}%<extra></extra>'
        ),
        secondary_y=False
    )
    
    # 5. Выделение PIT периода
    stress_indices = [
        i for i, row in data_df.iterrows() 
        if Period(int(row['year']), int(row['quarter'])) in pit_periods
    ]
    
    if stress_indices:
        # Затененная область для PIT периода
        fig.add_vrect(
            x0=min(stress_indices) - 0.3,
            x1=max(stress_indices) + 0.3,
            fillcolor='purple',
            opacity=0.15,
            layer='below',
            line_width=0,
            annotation_text="PIT период",
            annotation_position="top left",
            annotation_font_size=10
        )
        
        # Точки PIT периода
        pit_data = data_df.iloc[stress_indices]
        pit_quarters = pit_data.apply(lambda row: f"{int(row['year'])} Q{int(row['quarter'])}", axis=1).tolist()
        pit_rates = pit_data['quarterly_dr'].tolist()
        
        fig.add_trace(
            go.Scatter(
                x=pit_quarters,
                y=pit_rates,
                mode='markers',
                name='PIT период',
                marker=dict(
                    size=12,
                    color='purple',
                    symbol='diamond',
                    line=dict(width=2, color='white')
                ),
                hovertemplate='<b>PIT период</b><br>%{x}<br>Уровень: %{y:.2f}%<extra></extra>'
            ),
            secondary_y=False
        )
        
        # Линия PIT
        fig.add_trace(
            go.Scatter(
                x=quarters_labels,
                y=[pit_level] * len(quarters_labels),
                mode='lines',
                name=f'PIT: {pit_level:.2f}%',
                line=dict(color='purple', width=2, dash='dot'),
                hovertemplate=f'PIT: {pit_level:.2f}%<extra></extra>'
            ),
            secondary_y=False
        )
    
    # 6. Метрики в левом верхнем углу
    pit_ttc_ratio = pit_level / ttc_level if ttc_level > 0 else 0
    pit_uplift = (pit_level - ttc_level) / ttc_level * 100 if ttc_level > 0 else 0
    
    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        text=(
            f"<b>📊 Метрики калибровки</b><br>"
            f"<b>TTC:</b> {ttc_level:.2f}%<br>"
            f"<b>PIT:</b> {pit_level:.2f}%<br>"
            f"<b>PIT/TTC:</b> {pit_ttc_ratio:.2f}x<br>"
            f"<b>Uplift:</b> {pit_uplift:+.1f}%"
        ),
        showarrow=False,
        font=dict(size=10, family='Arial'),
        align="left",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="black",
        borderwidth=1,
        borderpad=4
    )
    
    # Настройка макета
    fig.update_layout(
        title={
            'text': 'Калибровка модели кредитного риска',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        xaxis=dict(
            title='Квартал',
            tickangle=45,
            showgrid=True,
            gridwidth=0.5,
            gridcolor='rgba(200, 200, 200, 0.3)'
        ),
        template='plotly_white',
        hovermode='x unified',
        height=500,
        width=1200,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="gray",
            borderwidth=1
        ),
        margin=dict(l=60, r=60, t=80, b=60)
    )
    
    # Настройка осей Y
    fig.update_yaxes(
        title_text="Уровень дефолта (%) / ВВП отклонение",
        secondary_y=False,
        showgrid=True,
        gridwidth=0.5,
        gridcolor='rgba(200, 200, 200, 0.3)'
    )
    
    if 'total_clients' in data_df.columns:
        fig.update_yaxes(
            title_text="К-во компаний",
            secondary_y=True,
            showgrid=False,
            range=[0, max(data_df['total_clients']) * 1.15]
        )
    
    # Добавляем базовую линию 0 для ВВП
    fig.add_hline(
        y=0,
        line_dash="dot",
        line_color="gray",
        line_width=1,
        opacity=0.5,
        secondary_y=False
    )
    
    # Сохранение
    if save_path:
        html_path = save_path if save_path.endswith('.html') else save_path.replace('.png', '.html')
        fig.write_html(html_path)
        print(f"📊 График сохранен: {html_path}")
    
    fig.show()
    
    return fig


In [22]:
chart_data = defaults_with_gdp.copy()
chart_data['quarterly_dr'] = chart_data['quarterly_dr'] * 100

In [23]:
chart = create_calibration_chart(
    chart_data,
    ttc_level*100,
    pit_level*100,
    pit_periods,
    save_path='calibration_chart.html'
)

📊 График сохранен: calibration_chart.html


### Сохранение результатов

In [24]:
CONFIG['calibration_params']['annual_drs'] = annual_drs
CONFIG['calibration_params']['central_tendency']['PIT'] = pit_level
CONFIG['calibration_params']['central_tendency']['TTC'] = ttc_level

In [25]:
# Записываем обратно
with open('../CONFIGS.yaml', 'w') as file:
    yaml.dump(CONFIG, file)